In [63]:
# !pip install requests-html requests pandas

  Using cached pandas-1.4.2-cp310-cp310-win_amd64.whl (10.6 MB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
  Using cached numpy-1.22.4-cp310-cp310-win_amd64.whl (14.7 MB)


In [66]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [5]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = 'python'
query_filter = "Votes"
url = f'{base_url}{tag}?tab={query_filter}'
url

'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [50]:
r = requests.get(url)
html_str = r.text
html = HTML(html=html_str)

In [8]:
# html = HTML(html=html_str)

In [24]:
question_summaries = html.find('.s-post-summary') #seleium inaczej
print(question_summaries)

<Element 'div' id='question-summary-231767' class=('s-post-summary', 'js-post-summary') data-post-id='231767' data-post-type-id='1'>


In [11]:
print(question_summaries[0].text)

12063 votes
48 answers
2.9m views
What does the "yield" keyword do?
What is the use of the yield keyword in Python? What does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...
python iterator generator
Alex. S.
136k
asked Oct 23, 2008 at 22:21


In [12]:
columns = ['votes', 'num_answers', 'views', 'question', 'short_desc', 'tags', 'user', 'user_details', 'date']

In [20]:
# this_row = list(question_summaries[0].text.split('\n'))
# this_row

In [18]:
# len(this_row) == len(columns)

In [17]:
# row_data = dict(zip(columns, this_row))
# row_data

In [44]:
key_names = ['question', 'votes', 'tags']
classes_nedded = ['.s-link', '.s-post-summary--stats-item', '.tags'] #gdy klasy się zmienią to cay kod się zmieni
this_question_element = question_summaries[0]
this_question_element.find('.s-link', first=True).text
this_question_element.find('.s-post-summary--stats-item', first=True).text.replace(' votes', '') #tutuaj można regexy

'12063'

In [47]:
def clean_scraped_data(text, keyname=None):
    if keyname == 'votes':
        return text.replace(' votes', '')
    return text

In [48]:
datas = []

for q_el in question_summaries:
    question_data = {}
    for i, _class in enumerate(classes_nedded):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
    datas.append(question_data)

In [49]:
datas[0]

{'question': 'What does the "yield" keyword do?',
 'votes': '12063',
 'tags': 'python iterator generator'}

In [52]:
def parse_tagged_page(html):
    question_summaries = html.find('.s-post-summary')
    key_names = ['question', 'votes', 'tags']
    classes_nedded = ['.s-link', '.s-post-summary--stats-item', '.tags'] #gdy klasy się zmienią to cay kod się zmieni
    datas = []
    for q_el in question_summaries:
        question_data = {}
        for i, _class in enumerate(classes_nedded):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
        datas.append(question_data)
    return datas


In [53]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [60]:
def scrape_tag(tag='python', query_filter = "Votes", page=1, max_pages=50, pagesize=25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f'{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}'
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas

In [62]:
datas = scrape_tag(tag='python')

In [64]:
len(datas)

2500

In [69]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,"What does the ""yield"" keyword do?",12063,python iterator generator
1,"What does if __name__ == ""__main__"": do?",7514,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,7323,python operators conditional-operator
3,What are metaclasses in Python?,6801,python oop metaclass python-class python-datam...
4,How do I check whether a file exists without e...,6651,python file file-exists


In [71]:
df.shape

(2500, 3)

In [73]:
df.to_csv('python.csv', index=False)